In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from pathlib import Path
import os
from pyxem.utils.expt_utils import investigate_dog_background_removal_interactive
from skimage import filters

from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

from orix.io import load, save

In [ ]:
#Load the data

folder = "/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/SPED"

file = "SPED_640x383x12_20x20_9p26x9p26nm_ScanRot20deg_CL12cm_spot0p5nm_alpha5_NBD" \
                                                        "_10ms_1deg_fixed_scan.zspy"
filepath = os.path.join(folder, file)
org_data = hs.load(filepath, lazy = True)

In [ ]:
#Inspect metadata

org_data.metadata

In [ ]:
#Plot the data for initial inspection
#Template matching on whole dataset can be slow.
#Speed is increased by only using every fourth pixel

#work_data = org_data.inav[::4,::4]

work_data = org_data
work_data.plot()

In [ ]:
#Set parameters for center direct beam

center_direct_beam_dict = {
    "method" : "blur",
    "half_square_width" : 10,
    "sigma" : 3
}
work_data.metadata.add_node("center_direct_beam_kwargs")
work_data.metadata.center_direct_beam_kwargs =  center_direct_beam_dict

#Center the beam
work_data.center_direct_beam(**work_data.metadata.center_direct_beam_kwargs)

In [ ]:
#investigate parameters for difference of gaussian background subtraction

gauss_stddev_maxs = np.arange(2, 12, 0.2) # min, max, step
gauss_stddev_mins = np.arange(1, 4, 0.2) # min, max, step


dp_test_area = work_data.inav[30,20]
investigate_dog_background_removal_interactive(dp_test_area,
                                               gauss_stddev_maxs,
                                               gauss_stddev_mins)

In [ ]:
#Set difference of background subtraction parameters

diff_background_removal_dict = {
    "method" : "difference of gaussians",
    "min_sigma":3,
    "max_sigma":6
}
work_data.metadata.add_node("subract_diff_background_kwargs")
work_data.metadata.subract_diff_background_kwargs = diff_background_removal_dict

In [ ]:
#Do preprocessing on data

def threshold_minimum(image, minimum=0.1):
    image[image <= minimum] = 0.
    return image


work_data = work_data.subtract_diffraction_background(
                    **work_data.metadata.subract_diff_background_kwargs)
                                            
work_data.map(filters.gaussian, sigma=0.5)
work_data.map(threshold_minimum, minimum = 1)
work_data.map(filters.gaussian, sigma=0.5)
work_data.map(filters.gaussian, sigma = 1.3)
work_data.map(threshold_minimum, minimum = 0.3)

In [ ]:
#plot to check preprocessing was succesful
work_data.plot(norm = 'symlog', cmap = 'inferno_r')

In [ ]:
#Generate library of simulated diffraction patterns

from Utils.GetDiffLibrary import GetDiffLibrary

diffraction_calibration = 0.00952354965
half_radius = 80
diff_lib = GetDiffLibrary(diffraction_calibration, 
                        12,
                        half_radius,
                        resolution=0.3,
                        make_new=True,
                        grid_cub = None, 
                        minimum_intensity=0.0015, 
                        max_excitation_error=0.0125,
                        precession_angle =0.3 
                        )
simulations = diff_lib["ErMnO3"]["simulations"]
orientations = diff_lib["ErMnO3"]["orientations"]

In [ ]:
#Do template matching on the data

def log_func(x):
    return(np.log10(x + 0.01))

    
result, phasedict = iutls.index_dataset_with_template_rotation(
                                                work_data,
                                                diff_lib,
                                                n_best = 2,
                                                intensity_transform_function=log_func,
                                                normalize_images = True,
                                                )

In [ ]:
#convert template matching results to crystal map for saving

xmap = iutls.results_dict_to_crystal_map(result,phasedict)

#Set space group symmetry for ErMnO3

xmap.phases[0].space_group = 185


In [ ]:
#results_dict_to_crystal_map() looses step size information. Needs to be recalibrated

def recalibrate_stepsize(xmap, stepsize = 6.92):
    y, x = np.indices(xmap.shape, dtype="float")
    x *= stepsize
    y *= stepsize
    xmap._x = x.ravel()
    xmap._y = y.ravel()
    xmap.scan_unit = "nm"

    return xmap

xmap = recalibrate_stepsize(xmap)

In [ ]:
# convert orientations to X and Y direction of scan
# The rotation needs to be calibrated for each dataset

from orix.quaternion import Rotation
from orix.crystal_map import CrystalMap

def apply_inplane_rotation(xmap, rotation = 56.54):
    rots_euler = xmap.rotations.to_euler()
    rots_euler += np.array([np.deg2rad(rotation),0,0])
    rot = Rotation.from_euler(rots_euler)
    phaseid = xmap.phase_id
    x = xmap.x
    y = xmap.y
    z = xmap.z
    prop = xmap.prop
    scan_unit = xmap.scan_unit
    is_in_data = xmap.is_in_data
    phaselist = xmap.phases

    return CrystalMap(rotations = rot,
                    phase_id = phaseid,
                    x = x,
                    y = y,
                    z = z,
                    prop = prop,
                    scan_unit = scan_unit,
                    is_in_data = is_in_data,
                    phase_list=phaselist)

xmap = apply_inplane_rotation(xmap)

In [ ]:
#save files as .hdf5 and .ang for later analysis

save('ErMnO3_1_deg.hdf5',xmap) 
save('ErMnO3_1_deg.ang',xmap) 